In [ ]:
from segmentation.scr.utils.utils import set_seed
from segmentation.scr.utils.metrics import dice_coef
from segmentation.models.smp import get_pretrained_model
from segmentation.config import CFG
import torch
import torch.nn as nn
import numpy as np
from segmentation.scr.inference.tta import tta

from segmentation.scr.train_model.data_loader import *
from segmentation.scr.utils.utils import set_seed, resize_to_size
from segmentation.scr.train_model.evaluate_dataset import evaluate_dataset
from segmentation.scr.inference import tta, create_tillings
from tqdm import tqdm



In [ ]:
model_my_pc= get_pretrained_model(path_to_model='weight\mobnet_my_pc.pth', train_parallel=False, CFG=CFG)


In [ ]:
path_img_dir="data\\train\\kidney_3_sparse\\images"
path_lb_dir="data\\train\\kidney_3_dense\\labels"
img_path, lb_path = create_img_lb_paths(path_img_dir = path_img_dir, path_lb_dir = path_lb_dir)

In [ ]:
class Data_loader_inference(Dataset):
    def __init__(self,img_path,lb_path):
        self.img_paths=img_path
        self.lb_path = lb_path
        h_m, w_m = 0,0
        for path in self.img_paths:
            img=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
            h, w  = img.shape
            h_m = max(h, h_m)
            w_m = max(w, w_m)
        self.img_size = (h_m, w_m)
        
        
    
    def __len__(self):
        return len(self.lb_path)- CFG.in_chans
        #return len(self.paths)
    
    def __getitem__(self,index):
        paths_img  = self.img_paths[index : index +  CFG.in_chans]
        path_lb  = self.lb_path[index + CFG.in_chans // 2]
        
        images = []

        for path_img in paths_img:
            img=cv2.imread(path_img,cv2.IMREAD_GRAYSCALE)
            h, w  = img.shape
            img=torch.from_numpy(img).to(torch.uint8)
            if h < self.img_size[0] or w < self.img_size[1]:
                img = resize_to_size(img=img, image_size=self.img_size)
            images.append(img)
            
        label = cv2.imread(path_lb,cv2.IMREAD_GRAYSCALE)
        label=torch.from_numpy(label!=0).to(torch.uint8)*255
        if h < self.img_size[0] or w < self.img_size[1]:  
            label = resize_to_size(img=label, image_size=self.img_size)
        images= torch.stack(images)
      

        return images, label

In [ ]:
loader_inf = Data_loader_inference(img_path, lb_path)

In [ ]:
val_loader = DataLoader(loader_inf, batch_size=1, shuffle=False, )

In [ ]:
len(val_loader)

In [ ]:
def predict_images(model, loader, use_tta = False, overlap_pct = 10):
    model.eval()
    timer = tqdm(range(len(loader)))
    val_scores = 0
    scores = []
    for i, (images, mask) in enumerate(loader):
        tills, indexs = create_tillings.create_tillings(images=images, overlap_pct=overlap_pct )
        till_shape = len(tills)
        tills = torch.cat(tills)
        tills =tills.to(torch.float32)
        #tills  = filter_noise(tills)
        tills = norm_with_clip(tills .reshape(-1, *tills.shape[2:])).reshape(tills.shape)
        with torch.no_grad():
            if not use_tta:
                pred = model(tills.cuda().to(torch.float32))
                pred = pred.cpu()
                pred = pred.sigmoid()
            else:
                pred = tta.tta(model=model, x=tills.cuda().to(torch.float32))
                pred = pred.cpu()
        pred = pred.reshape(till_shape, CFG.input_size, CFG.input_size)
        mask_pred = torch.zeros_like(mask,dtype=torch.float32)
        mask_count = torch.zeros_like(mask,dtype=torch.float32)
        
        for j,(y1,y2,x1,x2) in enumerate(indexs):
            mask_pred[...,y1:y2, x1:x2] += pred [j]
            mask_count[...,y1:y2, x1:x2] += 1
 
        mask_pred /= mask_count
        #print(mask_pred.shape, mask.shape)
        score = dice_coef(mask_pred.detach(), mask > 0, from_logits= False)
        scores.append(score.detach().cpu().item())
        val_scores = (val_scores * i + score.detach().cpu().item()) / (i + 1)
       
        timer.set_description(f"eval--> score:{val_scores:.4f}")
        timer.update()
        

    timer.close()
    return scores, val_scores

    

In [104]:
scores_no, val_scores = predict_images(model = model_my_pc, loader = val_loader, use_tta = True)

In [ ]:
import scipy
scipy.stats.ttest_rel(a=scores_yes, b=scores_no)#,alternative='greater')

In [ ]:
np.mean(scores_yes)

In [ ]:
np.mean(scores_no)

In [ ]:
scores_no

In [ ]:
images, masks = batch
images = (min_max_normalization(images.to(torch.float16)[None])[0]*255).to(torch.uint8)
tills, indexs = create_tillings(images=images, overlap_pct=10)
till_shape = len(tills)
tills = torch.cat(tills)
tills =tills.to(torch.float32)
tills = norm_with_clip(tills)

In [ ]:
model_my_pc.eval()
with torch.no_grad():
    ans= tta(model=model_my_pc, x=tills.cuda().to(torch.float32))
ans = ans.cpu()
#ans = ans.sigmoid()

In [ ]:
ans = ans.reshape(till_shape, 2, CFG.input_size, CFG.input_size)

In [ ]:
mask_pred = torch.zeros_like(masks,dtype=torch.float32)
mask_count = torch.zeros_like(masks,dtype=torch.float32)
for i,(y1,y2,x1,x2) in enumerate(indexs):
    mask_pred[...,y1:y2, x1:x2] += ans[i]
    mask_count[...,y1:y2, x1:x2] += 1
 
mask_pred /= mask_count

In [ ]:
dice_coef(mask_pred[1], masks > 0, from_logits= False, thr=0.5)

In [ ]:
import scipy
scipy.stats.ttest_rel(a=scores_my, b=scores_2)